<a href="https://colab.research.google.com/github/YoonTae-Hwang/trading_repository/blob/master/3_Backtrader_Qucistart_Guide_ANS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install backtrader
from google.colab import files
import pandas as pd
import io

upload = files.upload()

for i in upload.keys():
  print("User uploaded file  {name} with length {length} bytes".format(name = i, length = len(upload[i])))


BTC = pd.read_csv(io.StringIO(upload["BTC-USD.csv"].decode("utf-8")),
                  skiprows = 0,
                  parse_dates = True,
                  index_col = 0)
BTC

     |████████████████████████████████| 419kB 2.7MB/s 


Saving BTC-USD.csv to BTC-USD.csv
User uploaded file  BTC-USD.csv with length 146474 bytes


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-04-10,243.694000,243.694000,232.770996,236.072006,236.072006,2.888200e+07
2015-04-11,236.016006,239.537003,234.175003,236.552002,236.552002,1.636520e+07
2015-04-12,236.535004,237.727997,233.494995,236.153000,236.153000,1.238790e+07
2015-04-13,235.949997,236.934998,221.996002,224.587006,224.587006,3.118180e+07
2015-04-14,224.759003,224.975998,216.322998,219.158997,219.158997,3.171900e+07
...,...,...,...,...,...,...
2020-04-06,6788.049805,7271.781250,6782.889648,7271.781250,7271.781250,4.689690e+10
2020-04-07,7273.644043,7427.939453,7136.714355,7176.414551,7176.414551,4.424348e+10
2020-04-08,7179.283203,7356.223633,7153.305664,7334.098633,7334.098633,3.756325e+10


- 전략을 세우기 위해서는 다음과 같이 class를 만듭니다. log function은 이전에 이미 설명하였기 때문에 생략합니다.
- 이전에는 원하는 가격에 살 수 있다고 생각했습니다. 그런데 실제로 주문을 넣었을때 이보다 낮은가격이나 높은가격에 살 수도 있습니다. 

1. ``` python
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None 
        self.buyprice = None
        self.buycomm = None
```

- self.order = None 라고 두어서 주문이 진행되는 상태를 추적해봅시다.[Link](https://www.backtrader.com/docu/order/)
- self.order, self.buycomm으로 우리가 주문상태외에에도 주문가격과 수수료를 추적해봅시다. None이라고 붙인이유는 우리가 앞으로 notify_order function에서 사용하기 위함입니다.

***

2. ``` python
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None
```
- 우선, 첫번째 조건문은 다음과 같습니다. 우리가 정상적으로 주문을 넣게되면 주문상태는 주문이 들어가거나 주문이 Accepted되게됩니다. 
- 다음 조건문을 살펴보면, 주문이 성공적으로 진행되면 우리들은 사거나 팔 수 있습니다. 만약 사는 경우에 **BUY EXECUTED 주식1개의가격, Cost : 주식 n개 * 주식가격, comm : 수수료 **을 출력합니다. [link](https://community.backtrader.com/topic/1038/order-execution-price/4)
- self.bar_executed = len(self)  을 통해서 우리가 기본값인 5분봉이내로 주문을 집행합니다. [link](https://randlow.github.io/posts/trading/trading-custom-backtrader/)
- 만약 거절된다면 Order Canceled/Margin/Rejected 라고 출력합니다.

***

3. ``` python
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

```
- 조건문은 우리가 백테스팅을 한 시점까지(today) 계속해서 아래 문장을 출력하도록합니다.
- pnl은 Gross 이고, pnlcomm 순수익입니다. 

4. ``` python
    def next(self):
#        self.log('Close, %.2f' % self.dataclose[0])

        if self.order:
            return

        if not self.position:
            if self.dataclose[0] < self.dataclose[-1]:
                    if self.dataclose[-1] < self.dataclose[-2]:
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])
                        self.order = self.buy(size = 100)

        else:

            if len(self) >= (self.bar_executed + 5):
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell(size = 100)
```

- 여기서 눈여겨봐야할점은 self.order = self.buy(size = 100) 이 부분이다.
- buy(size = 100)은 100개의 주식을 구입한다는것인데 이 부분이 order로 가서 주문이 들어간다. *우리는 진행상황을 자세하게 알고싶다!
-  if len(self) >= (self.bar_executed + 5): [이부분 추후업로드]

5. ``` python
    cerebro.broker.setcommission(commission=0.001)

```
- 마지막으로 실행코드에서 commision 은 수수료이다.


In [14]:
import datetime 
import backtrader as bt

class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close

        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(  'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % (order.executed.price, order.executed.value, order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % (order.executed.price,order.executed.value, order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
#        self.log('Close, %.2f' % self.dataclose[0])
        if self.order:
            return

        if not self.position:
            if self.dataclose[0] < self.dataclose[-1]:
                    if self.dataclose[-1] < self.dataclose[-2]:
 #                       self.log('BUY CREATE, %.2f' % self.dataclose[0])
                        self.order = self.buy()

        else:

            if len(self) >= (self.bar_executed + 5):
 #               self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

if __name__ == "__main__":
  cerebro = bt.Cerebro()
  
  cerebro.addstrategy(TestStrategy)

  data = bt.feeds.PandasData(dataname = BTC)
  cerebro.adddata(data)

  cerebro.broker.setcash(15000.0)

  cerebro.broker.setcommission(commission=0.01)

  print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
  
  cerebro.run()

  print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 15000.00
2015-04-14, BUY EXECUTED, Price: 224.76, Cost: 224.76, Comm 2.25
2015-04-20, SELL EXECUTED, Price: 222.61, Cost: 224.76, Comm 2.23
2015-04-20, OPERATION PROFIT, GROSS -2.15, NET -6.62
2015-04-26, BUY EXECUTED, Price: 226.41, Cost: 226.41, Comm 2.26
2015-05-02, SELL EXECUTED, Price: 232.34, Cost: 226.41, Comm 2.32
2015-05-02, OPERATION PROFIT, GROSS 5.93, NET 1.34
2015-05-06, BUY EXECUTED, Price: 236.25, Cost: 236.25, Comm 2.36
2015-05-12, SELL EXECUTED, Price: 242.15, Cost: 236.25, Comm 2.42
2015-05-12, OPERATION PROFIT, GROSS 5.90, NET 1.11
2015-05-14, BUY EXECUTED, Price: 236.21, Cost: 236.21, Comm 2.36
2015-05-20, SELL EXECUTED, Price: 231.89, Cost: 236.21, Comm 2.32
2015-05-20, OPERATION PROFIT, GROSS -4.32, NET -9.01
2015-05-31, BUY EXECUTED, Price: 233.13, Cost: 233.13, Comm 2.33
2015-06-06, SELL EXECUTED, Price: 225.01, Cost: 233.13, Comm 2.25
2015-06-06, OPERATION PROFIT, GROSS -8.13, NET -12.71
2015-06-19, BUY EXECUTED, Price: 249.04, Cost: 2